In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from tqdm import tqdm


plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用SimHei字体
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

import matplotlib.dates as mdates

def find_csv_files(root_dir, file_name):
    """
    在给定的根目录下查找所有的csv文件，并返回它们的相对路径。
    """
    
    csv_files = []

    # 使用os.walk遍历目录树
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # 如果当前目录是__MACOSX，则跳过
        if '__MACOSX' in dirpath.split(os.sep):
            continue

        for filename in filenames:
            if filename.endswith(file_name):
                # 使用os.path.join连接目录和文件名，然后使用os.path.relpath获取相对路径
                relative_path = os.path.relpath(os.path.join(dirpath, filename), root_dir)
                csv_files.append('2023因子日历数据\\'+relative_path)

    return csv_files


In [2]:
#Coefficient_of_Variation_of_Turnover
csv_de_name = 'liquidity_factor.csv'
#读取同一个领域的所有值，然后进行合并
csv_files_list = find_csv_files('2023因子日历数据',csv_de_name)
df = pd.DataFrame()
for idx,file_name in enumerate(csv_files_list):
    df_tmp = pd.read_csv(file_name,encoding='gbk')
    df = pd.concat([df,df_tmp],axis=0)

#检查 df 的各列的缺失比例
df.isnull().sum()/df.shape[0]
#保留缺失比例在10%以下的因子
df = df.loc[:, df.isnull().sum()/df.shape[0] < 0.1]
df_PCTCHANGE = pd.read_csv('PCTCHANGE.csv')
#将df和df_PCTCHANGE按照trade_date与sec_code	合并
df = pd.merge(df,df_PCTCHANGE,on=['trade_date','sec_code'],how='right')
del df_tmp,df_PCTCHANGE


# Assuming df is your DataFrame
excluded_columns = ['trade_date', 'sec_code', 'PCTCHANGE', 'next_day_close']
columns_list = [col for col in df.columns if col not in excluded_columns]


In [ ]:
for factor in tqdm(columns_list):
    #抽取df中的'trade_date', 'sec_code', 'PCTCHANGE', 'next_day_close'，Information_Discreteness列
    select_columns = ['trade_date', 'sec_code', 'PCTCHANGE']
    select_columns.append(factor)
    df_tmp = df[select_columns].copy()
    
    # 确保 trade_date 是 datetime 格式，并且数据是按 trade_date 升序排列
    df_tmp['trade_date'] = pd.to_datetime(df_tmp['trade_date'])
    df_tmp = df_tmp.sort_values(by='trade_date')

    df_tmp['next_day_close'] = df_tmp.groupby('sec_code')['PCTCHANGE'].shift(-1)
    df_tmp.dropna(inplace=True)
    # 计算 RANKIC

    ic_values = df_tmp.groupby('trade_date').apply(lambda x: spearmanr(x[factor], x['next_day_close'])[0]).replace([np.inf, -np.inf], np.nan).dropna()

    #胜率
    df_tmp['predict'] = df_tmp[factor].apply(lambda x: 1 if x > 0 else -1)
    df_tmp['win'] = (df_tmp['predict'] * df_tmp['next_day_close'] > 0).astype(int)
    win_rate = df_tmp['win'].mean()

    # ICIR
    icir = ic_values.mean() / ic_values.std()

    #10分组多空收益

    # 假设 df_tmp 已经被定义
    # 从 trade_date 中提取年份和月份信息
    df_tmp['year'] = df_tmp['trade_date'].dt.year
    df_tmp['month'] = df_tmp['trade_date'].dt.month

    # 使用 groupby 方法按照 sec_code、年份和月份对数据进行分组
    grouped = df_tmp.groupby(['sec_code', 'year', 'month'])

    # 对每个分组计算因子的均值以及涨跌幅的连乘
    result = grouped.agg({
        factor: 'mean',
        'PCTCHANGE': lambda x: (x/100 + 1).prod() - 1
    }).reset_index()
    # 重命名列
    result.rename(columns={'PCTCHANGE': 'monthly_return'}, inplace=True)
    #将result中的year与month列拼成时间，格式为datetime
    result['year'] = result['year'].astype(str)
    result['month'] = result['month'].astype(str)
    result['date'] = result['year'] + '-' + result['month']
    result['date'] = pd.to_datetime(result['date'])
    result['future_ret'] = result.groupby('sec_code')['monthly_return'].shift(-1)

    # 确保 trade_date 是 datetime 格式，并且数据是按 trade_date 升序排列

    result = result.sort_values(by='date')

    long_returns = []
    short_returns = []
    returns = []
    dates = []

    # 对每一个交易日
    for date, group in result.groupby('date'):
        
        # 基于因子值对股票进行排序
        sorted_group = group.sort_values(by=factor, ascending=False)
        n = len(sorted_group)
        
        # 计算前10%和后10%的平均收益
        long_return = sorted_group.head(int(n*0.1))['future_ret'].mean()
        short_return = sorted_group.tail(int(n*0.1))['future_ret'].mean()
        
        # 计算策略的净收益率
        net_return = long_return - short_return
        
        # 保存结果到相应的列表
        long_returns.append(long_return)
        short_returns.append(short_return)
        returns.append(net_return)
        dates.append(date)

    # 如果需要，您可以将这些列表转换为Pandas Series，其中日期作为索引：
    long_returns_series = pd.Series(long_returns, index=dates)
    short_returns_series = pd.Series(short_returns, index=dates)
    returns_series = pd.Series(returns, index=dates)

    long_returns_series = long_returns_series[:-1]
    short_returns_series = short_returns_series[:-1]
    returns_series = returns_series[:-1]

    print(factor)
    print(f"IC: {ic_values.mean()*100:.2f}%")
    print(f"胜率: {max(1-win_rate,win_rate)*100:.2f}%")
    print(f"ICIR: {icir:.2f}")
    annual_return = ((1 + pd.Series(returns_series)).prod())**(12/len(returns)) - 1
    print(f"多空对冲年化收益: {abs(annual_return)*100:.2f}%")

    ic_values.cumsum().plot(figsize=(15, 5), grid=True, legend=True).set_title(factor+'的累计IC图', fontsize=20)

    #将returns_df的列名分别设置为long_returns、short_returns、returns
    if annual_return<0:
        #将long_returns_series、short_returns_series、returns_series三个序列合并为一个DataFrame
        returns_df = pd.concat([(1+long_returns_series).cumprod(), (1+short_returns_series).cumprod(), (1-returns_series).cumprod()], axis=1)
        returns_df.columns = [ 'short_returns', 'long_returns','returns']   
    else:
        returns_df = pd.concat([(1+long_returns_series).cumprod(), (1+short_returns_series).cumprod(), (1+returns_series).cumprod()], axis=1)
        returns_df.columns = [ 'long_returns','short_returns', 'returns']  
    #画图
    returns_df.plot(figsize=(15, 5), grid=True, legend=True).set_title(factor+'因子 10 分组多空对冲净值走势', fontsize=20)
    plt.show()
    print("##############################")

